In [75]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [76]:
train_df =pd.read_pickle('model_datasets/train_model.pbz2')
valid_df=pd.read_pickle('model_datasets/valid_model.pbz2')
test_df=pd.read_pickle('model_datasets/test_model.pbz2')

In [77]:
train_df=train_df[train_df.fips.str.startswith('8')]
valid_df=valid_df[valid_df.fips.str.startswith('8')]
test_df=test_df[test_df.fips.str.startswith('8')]

In [78]:
def shape_data(df,cols):
    dates=df['date'].unique()
    fips=df['fips'].unique()
    lista=[]
    for date in tqdm(dates):
        date_lst=[]
        for fip in fips:
            feat_lst=df.loc[(df['date']==date) & (df['fips']==fip)][cols].to_numpy().flatten()
            date_lst.append(feat_lst)
        lista.append(date_lst)
    array=np.array(lista)
    return array

In [79]:
def prep_array(df):

    df['date']=df['date'].apply(lambda x:x.timestamp())
    df['date']=df['date'].astype('float32')

    df_redu=pd.get_dummies(df,columns=['score'])
    ordem_cols=['date','fips']
    new_cols = ordem_cols + df_redu.drop(ordem_cols,axis=1).columns.to_list()
    df_redu=df_redu[new_cols]
    out_df=pd.get_dummies(df[['date','fips','score']],columns=['score'])
    out_df[out_df.columns.to_list()[-6:]]=out_df[out_df.columns.to_list()[-6:]].shift(-6)
    #Os ultimoos 6 irei colocar como sendo classe 0, é a classe mais comum e como a fração nula 
    #é muito pequena 6/6656 no pior dos casosnão deve afetar a perfomance do modelo
    out_df[out_df.columns.to_list()[-5:]]=out_df[out_df.columns.to_list()[-5:]].fillna(0)
    out_df['score_0.0']=out_df['score_0.0'].fillna(1)
    
    cols= df_redu.columns.to_list()[2:]
    cols_out =out_df.columns.to_list()[2:]
    input_array=shape_data(df_redu,cols)
    output_array=shape_data(out_df,cols_out)
    return input_array,output_array


In [80]:
train_in_array,train_out_array = prep_array(train_df)
np.save('model_datasets/train_in_array.npy',train_in_array)
np.save('model_datasets/train_out_array.npy',train_out_array)

  0%|          | 0/887 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

In [81]:
valid_in_array,valid_out_array = prep_array(valid_df)
np.save('model_datasets/valid_in_array.npy',valid_in_array)
np.save('model_datasets/valid_out_array.npy',valid_out_array)

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

In [82]:
test_in_array,test_out_array = prep_array(test_df)
np.save('model_datasets/test_in_array.npy',test_in_array)
np.save('model_datasets/test_out_array.npy',test_out_array)

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]